This Jupyter Notebook is meant to be accessed via Google Colab and will be the main method to train models on Google Colab.

In [4]:
# primary importing code block to utilize the python files
import os
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, LeakyReLU, ZeroPadding2D, BatchNormalization, MaxPool2D
from pydub import AudioSegment   # main class from pydub package used to upload mp3 into Python and then get a NumPy array
import IPython.display as ipd    # ability to play audio in Jupyter Notebooks if needed
import librosa as lb             # loads the librosa package
import librosa.display
from src.configs import *
from src.utils import MusicAlignedTab, create_FullSet_df, one_hot_encode, collapse_class, clean_labels
from src.dataset import Dataset

In [ ]:
mat = MusicAlignedTab('mookies_last_christmas')
df = mat.MAT

In [ ]:
df = clean_labels(df)
MusicAlignedTab.labels_summary(df)
df = collapse_class(df, keep_dynamics = False, keep_bells = False, keep_toms_separate = False, hihat_classes=1, cymbal_classes=1)
MusicAlignedTab.labels_summary(df)

In [ ]:
encode_df = one_hot_encode(df)
print(encode_df.columns)
encode_df.head()

In [ ]:
%time targets, target_dict = create_targets(S, encode_df)

In [ ]:
for idx, val in target_dict.items():
    print(f'total windows that {val} are labeled 1 = {np.count_nonzero(targets[idx,:,0])}')
print(f'total windows = {targets.shape[1]}')

In [ ]:
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot()
plt.imshow(targets[:,:,1])
plt.show()

In [ ]:
MusicAlignedTab.labels_summary(encode_df)

In [ ]:
targets = np.zeros((7, 10, 3), dtype=int)
print(targets.shape)
targets[:, 0, :]

In [ ]:
df.drop(columns = ['song slice', 'sample start'])[df.drop(columns = ['song slice', 'sample start']) != '-'].describe()

In [ ]:
mat.random_alignment_checker(['BD', 'SD', 'CC'], 14)

In [ ]:
librosa.display.specshow(S[:,:,0], sr=44100, x_axis='time', y_axis = 'mel')

### FullSet Testing

In [ ]:
def play(samples, sr=44100):
    '''
    Helper function just so I can type play(samples) to get the song output
    '''
    return ipd.Audio(samples, rate = sr)

In [ ]:
FullSet = create_FullSet_df(SONGS_PATH)
FullSet_clean = clean_labels(FullSet)
MusicAlignedTab.labels_summary(FullSet_clean)
FullSet_collapse = collapse_class(FullSet_clean, keep_dynamics = False, keep_bells = False, keep_toms_separate = False, hihat_classes=1, cymbal_classes=1)
MusicAlignedTab.labels_summary(FullSet_collapse)
FullSet_encoded = one_hot_encode(FullSet_collapse)

In [ ]:
tset = Dataset('train', FullSet_encoded)
vset = Dataset('val', FullSet_encoded)

In [ ]:
song_idx = 1

In [ ]:
%%time
song_df = tset.subset_df.loc[tset.song_list[song_idx]].copy()
song = np.vstack(song_df['song slice'].to_numpy()).T   # stacks the song slices back into a single numpy array of shape (channels, samples)
mono_song = lb.core.to_mono(song)
channels = [mono_song]              # channels is a list of either [mono_song] or [mono, L_song, R_song]
if INCLUDE_LR_CHANNELS:             # appending the LR channels to the channels variable
    channels.append(song[0,:])
    channels.append(song[1,:])
aug_channels = tset.augment_audio_cp(channels, tset.aug_comp, sr=SAMPLE_RATE)

In [ ]:
play(aug_channels[0])

In [ ]:
INCLUDE_LR_CHANNELS = False
%time spectrogram, target, target_dict = tset.preprocess_song(tset.song_list[7])

In [ ]:
print(spectrogram.shape)
print(target.shape)
print(target_dict)
tset.song_list[7]

In [ ]:
vset.subset_df

In [ ]:
labels = np.zeros((5,5))

In [ ]:
labels_df = vset.subset_df.loc['four_years'].drop(columns = ['sample start', 'song slice']).copy()

In [ ]:
labels_df

In [ ]:
np.stack([labels_df.loc[0].to_numpy() for _ in range(3)], axis = -1).shape

In [ ]:
targets = np.zeros((7, 15,3))

In [ ]:
%%time
save_spec = []
save_tar = []
for spec, tar in vset:
    save_spec.append(spec)
    save_tar.append(tar)
    print(f'shape of target = {tar.shape}')

In [ ]:
fy_spec = save_spec[0]
mb_spec = save_spec[1]
fy_tar = save_tar[0]
mb_tar = save_tar[1]

In [ ]:
numpysave_fp = 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Experimental/'

In [ ]:
np.save(numpysave_fp + 'fy_spec', fy_spec)
np.save(numpysave_fp + 'fy_tar', fy_tar)
np.save(numpysave_fp + 'm_spec', mb_spec)
np.save(numpysave_fp + 'mb_tar', mb_tar)

In [ ]:
np.savez(numpysave_fp + 'zip_uncomp', fy_spec, fy_tar, mb_spec, mb_tar)

In [ ]:
np.savez_compressed(numpysave_fp + 'zip_comp', fy_spec, fy_tar, mb_spec, mb_tar)

In [ ]:
fy_spec.shape

In [ ]:
fy_tar.shape

In [ ]:
one_chan = fy_spec[:,:,0]

In [ ]:
def expand_spectrogram(spectrogram, pre_context, post_context):
    '''
    Expands a 2D spectrogram into slices of the correct shape to be input into the model
    
    Args:
        spectrogram [np.array]:
        pre_context [int]:
        post_context [int]:
    
    Returns:
        np.array: 
    
    '''
    
    n_features, n_windows = spectrogram.shape
    
    input_width = 1 + pre_context + post_context
    min_value = np.min(spectrogram)
    
    expanded_spectrogram = np.full(shape = (n_windows, n_features, input_width), fill_value = min_value)# assign into this np.array
    
    for idx in range(n_windows):
        if idx - pre_context < 0:    # in a window where you would slice before the beginning
            start = pre_context-idx
            expanded_spectrogram[idx, :,start:] = spectrogram[:, 0:idx+post_context+1 ]
        elif idx + post_context+1 > n_windows: # in a window where you would slice past the end
            end = post_context+1 - (n_windows - idx)
            expanded_spectrogram[idx, :, :input_width-end] = spectrogram[:, idx-pre_context: n_windows ]
        else:    # in a "normal" middle window where you slice into the spectrogram normally
            expanded_spectrogram[idx, :,:] = spectrogram[:, idx-pre_context : idx+post_context+1]
    
    return expanded_spectrogram

In [ ]:
%%time
exp_spec = expand_spectrogram(one_chan, 5,5)
exp_spec[10000,0,:]

In [ ]:
exp_spec.

In [ ]:
exp_spec[:,:,0]

In [ ]:
one_chan.shape

## Testing of the train.py

In [1]:
%run train.py

subdirs = ['C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\ancient_tombs', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\best_of_me', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\boulevard_of_broken_dreams', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\cant_be_saved', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\face_down', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\family_tradition', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\fireworks_at_dawn', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\forever_at_last', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\four_years', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\garden_state', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\gunpowder', 'C:/Users/Thomas/Python Projects/Drum-Tabber-Support-Data/Songs\\hair_of_the_dog', 'C:/User

C:\ProgramData\Anaconda3\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


tab length = 2882     datatype: <class 'int'>
len(song_slices_tab_indexed) = 2882     datatype of object: <class 'list'>
song_slices_tab_indexed[0].shape = (3392, 2)     datatype of [0]: <class 'numpy.ndarray'>
np.array(song_slices_tab_indexed).shape = (2882,)
len(sample_start_list) = 2882
tab length = 1628     datatype: <class 'int'>
len(song_slices_tab_indexed) = 1628     datatype of object: <class 'list'>
song_slices_tab_indexed[0].shape = (6182, 2)     datatype of [0]: <class 'numpy.ndarray'>
np.array(song_slices_tab_indexed).shape = (1628,)
len(sample_start_list) = 1628
tab length = 1440     datatype: <class 'int'>
len(song_slices_tab_indexed) = 1440     datatype of object: <class 'list'>
song_slices_tab_indexed[0].shape = (7922, 2)     datatype of [0]: <class 'numpy.ndarray'>
np.array(song_slices_tab_indexed).shape = (1440,)
len(sample_start_list) = 1440
tab length = 2110     datatype: <class 'int'>
len(song_slices_tab_indexed) = 2110     datatype of object: <class 'list'>
song_s

---dataframe.describe() without blank_chars---
           tk     BD    SD    HH    RD    CC    C2   LT   MT   HT  CH   C3  SC
count   49917  11829  7551  5351  1718  3275  2331  994  391  298  97  290  26
unique      9      2     3     4     4     3     3    2    2    2   3    2   2
top         +      o     o     x     x     X     X    o    o    o   X    X   X
freq    12461  11399  6752  2907  1104  2016  1891  916  334  262  81  264  20

---Unique values and frequencies by column name---
        +      e      a     1     2     3     4   t   s
tk  12461  12459  12446  3206  3202  3199  2867  47  30

        -      o    O
BD  38088  11399  430

        -     o    g    O
SD  42366  6752  527  272

        -     x     X    o   g
HH  44566  2907  2188  230  26

        -     x    X    b   g
RD  48199  1104  476  113  25

        -     X     x  b
CC  46642  2016  1252  7

        -     X    x  b
C2  47586  1891  439  1

        -    o   O
LT  48923  916  78

        -    o   O
MT  49526  33

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x


...

C:\ProgramData\Anaconda3\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


Epoch: 2 Song 18/23, lr:0.000036, song_loss:0.693203
Dataset class __next__: preprocessing wolves_at_the_door
Epoch: 2 Song 19/23, lr:0.000034, song_loss:0.693193
Dataset class __next__: preprocessing the_kill
Epoch: 2 Song 20/23, lr:0.000032, song_loss:0.693198
Dataset class __next__: preprocessing boulevard_of_broken_dreams
Epoch: 2 Song 21/23, lr:0.000030, song_loss:0.693193
Dataset class __next__: preprocessing family_tradition


C:\ProgramData\Anaconda3\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


Epoch: 2 Song 22/23, lr:0.000028, song_loss:0.693193
Dataset class __next__: preprocessing hair_of_the_dog
Epoch: 2 Song  0/23, lr:0.000026, song_loss:0.693193
Dataset class __next__: preprocessing four_years
Dataset class __next__: preprocessing misery_business


Epoch:  2 val_loss:0.738158 


Starting Epoch 3/4
Dataset class __next__: preprocessing sow
Epoch: 3 Song  1/23, lr:0.000024, song_loss:0.693193
Dataset class __next__: preprocessing cant_be_saved
Epoch: 3 Song  2/23, lr:0.000022, song_loss:0.693181
Dataset class __next__: preprocessing hair_of_the_dog


KeyboardInterrupt: 

In [29]:
int1 = 0
int2 = 100
Song = 1
lr= 0.00023453
song_l = 234.4566

In [33]:
print('Epoch: {:2}/{} Song{:3}, lr:{:.6f}, song_loss:{:8.2f}'.format(int1, int2, Song, lr, song_l))

Epoch:  0/100 Song  1, lr:0.000235, song_loss:  234.46
